In [155]:
from datetime import datetime

from google.cloud import aiplatform
import os

In [156]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

In [157]:
PROJECT_ID

'qwiklabs-asl-04-76d2f9d7a5ae'

In [158]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin:


In [159]:
%%bash
ls /home/jupyter/.local/bin

__pycache__
dsl-compile
dsl-compile-v2
identify-cli
jsonschema
kfp
nodeenv
plasma_store
portserver.py
pre-commit
pre-commit-validate-config
pre-commit-validate-manifest
strip-hints
tb-gcp-uploader
tfx


In [160]:
import os

In [161]:
%%writefile ./Dockerfile
#FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest
FROM us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest
RUN pip install -U cloudml-hypertune

COPY . /code

WORKDIR /code

ENTRYPOINT ["python3", "-m", "trainer.task"]

Overwriting ./Dockerfile


In [162]:
%%bash
cat Dockerfile

#FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest
FROM us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-8:latest
RUN pip install -U cloudml-hypertune

COPY . /code

WORKDIR /code

ENTRYPOINT ["python3", "-m", "trainer.task"]


In [163]:
IMAGE_NAME = "rutu_image_ct_phase_contrast_kfp_3"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"
TRAINING_CONTAINER_IMAGE_URI


'gcr.io/qwiklabs-asl-04-76d2f9d7a5ae/rutu_image_ct_phase_contrast_kfp_3:latest'

In [164]:

SERVING_CONTAINER_IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-8:latest"

In [165]:
%%bash
ls

01_jay_contrast_pre_containerize_generic.ipynb
02_jay_contrast_container_kfp_pipeline .ipynb
Dockerfile
__pycache__
dist
pipeline_prebuilt_jay_3.py
rutu_capstone_trainer_5_1.egg-info
rutu_ct_pipeline_new1.json
setup.py
trainer


In [166]:
#os.chdir("/home/jupyter/asl-ml-immersion/notebooks/capstone_project/CapStone_Phase_Contrast/src/")
os.getcwd()

'/home/jupyter/rutuja_april5'

In [167]:
!gcloud builds submit --timeout 40m --tag $TRAINING_CONTAINER_IMAGE_URI .

Creating temporary tarball archive of 18 file(s) totalling 248.1 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-asl-04-76d2f9d7a5ae_cloudbuild/source/1680875027.835113-7b8ccec297f246e688addef34d3aff08.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-04-76d2f9d7a5ae/locations/global/builds/e84d7d7d-737f-4410-b295-d7beda20c43f].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/e84d7d7d-737f-4410-b295-d7beda20c43f?project=146339387885 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "e84d7d7d-737f-4410-b295-d7beda20c43f"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-04-76d2f9d7a5ae_cloudbuild/source/1680875027.835113-7b8ccec297f246e688addef34d3aff08.tgz#1680875028209516
Copying gs://qwiklabs-asl-04-76d2f9d7a5ae_cloudbuild/source/1680875027.835113-7b8ccec297f246e688addef34d3aff08.tgz#1680875028209516...
/ [1 files][ 50.3 KiB/ 50.3 KiB]                          

In [168]:
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-8:latest"
)

In [169]:
%%writefile ./pipeline_prebuilt_jay_3.py


"""Kubeflow CT Pipeline."""

import os

from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google_cloud_pipeline_components.aiplatform import (
    EndpointCreateOp,
    ModelDeployOp,
    ModelUploadOp,
)
from google_cloud_pipeline_components.experimental import (
    hyperparameter_tuning_job,
)
from google_cloud_pipeline_components.experimental.custom_job import (
    CustomTrainingJobOp,
)
from kfp.v2 import dsl

PIPELINE_ROOT = os.getenv("PIPELINE_ROOT")
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")

TRAINING_CONTAINER_IMAGE_URI = os.getenv("TRAINING_CONTAINER_IMAGE_URI")
SERVING_CONTAINER_IMAGE_URI = os.getenv("SERVING_CONTAINER_IMAGE_URI")
SERVING_MACHINE_TYPE = os.getenv("SERVING_MACHINE_TYPE", "n1-standard-16")

TRAINING_FILE_PATH = os.getenv("TRAINING_FILE_PATH")
VALIDATION_FILE_PATH = os.getenv("VALIDATION_FILE_PATH")

MAX_TRIAL_COUNT = int(os.getenv("MAX_TRIAL_COUNT", "30"))
PARALLEL_TRIAL_COUNT = int(os.getenv("PARALLEL_TRIAL_COUNT", "5"))
THRESHOLD = float(os.getenv("THRESHOLD", "0.6"))

PIPELINE_NAME = os.getenv("PIPELINE_NAME", "rutu-ct-pipeline-3")
BASE_OUTPUT_DIR = os.getenv("BASE_OUTPUT_DIR", PIPELINE_ROOT)
MODEL_DISPLAY_NAME = os.getenv("MODEL_DISPLAY_NAME", PIPELINE_NAME)





@dsl.pipeline(
    name=f"{PIPELINE_NAME}-kfp-pipeline",
    description="Kubeflow pipeline that tunes, trains, and deploys on Vertex",
    pipeline_root=PIPELINE_ROOT,
)
def create_pipeline():

    worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-highmem-16",
                "accelerator_type": "NVIDIA_TESLA_P100",
                "accelerator_count": 2,
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": TRAINING_CONTAINER_IMAGE_URI,
                "args": [
                    f"--train_data_path={TRAINING_FILE_PATH}",
                    f"--eval_data_path={VALIDATION_FILE_PATH}",
                    f"--output_dir={BASE_OUTPUT_DIR}",
                    "--hptune",
                ],
            },
        }
    ]
      

    metric_spec = hyperparameter_tuning_job.serialize_metrics(
        {"final_val_accuracy": "maximize"}
    )

    parameter_spec = hyperparameter_tuning_job.serialize_parameters(
        {
            "dropout_rate": hpt.DoubleParameterSpec(
                min=1.0e-3, max=4.0e-1, scale="log"
            ),
            "l2_regularization_lambda": hpt.DoubleParameterSpec(
                min=1.0e-3, max=4.0e-1, scale="log"
            ),
        }
    )

    hp_tuning_task = hyperparameter_tuning_job.HyperparameterTuningJobRunOp(
        display_name=f"{PIPELINE_NAME}-kfp-tuning-job",
        project=PROJECT_ID,
        location=REGION,
        worker_pool_specs=worker_pool_specs,
        study_spec_metrics=metric_spec,
        study_spec_parameters=parameter_spec,
        max_trial_count=MAX_TRIAL_COUNT,
        parallel_trial_count=PARALLEL_TRIAL_COUNT,
        base_output_directory=PIPELINE_ROOT,
    )

    trials_task = hyperparameter_tuning_job.GetTrialsOp(
        gcp_resources=hp_tuning_task.outputs["gcp_resources"]
    )

    best_hyperparameters_task = (
        hyperparameter_tuning_job.GetBestHyperparametersOp(
            trials=trials_task.output, study_spec_metrics=metric_spec
        )
    )
    
    
    # Construct new worker_pool_specs and
    # train new model based on best hyperparameters
    worker_pool_specs_task = hyperparameter_tuning_job.GetWorkerPoolSpecsOp(
        best_hyperparameters=best_hyperparameters_task.output,
        worker_pool_specs=[
            {
                "machine_spec": {"machine_type": "n1-highmem-16",
                                 "accelerator_type": "NVIDIA_TESLA_P100",
                                 "accelerator_count": 2},
                "replica_count": 1,
                "container_spec": {
                    "image_uri": TRAINING_CONTAINER_IMAGE_URI,
                    "args": [
                        f"--train_data_path={TRAINING_FILE_PATH}",
                        f"--eval_data_path={VALIDATION_FILE_PATH}",
                        f"--output_dir={BASE_OUTPUT_DIR}",
                    ],
                },
            }
        ],
    )

    training_task = CustomTrainingJobOp(
        project=PROJECT_ID,
        location=REGION,
        display_name=f"{PIPELINE_NAME}-kfp-training-job",
        worker_pool_specs=worker_pool_specs_task.output,
        base_output_directory=BASE_OUTPUT_DIR,
    )

    model_upload_task = ModelUploadOp(
        project=PROJECT_ID,
        display_name=f"{PIPELINE_NAME}-kfp-model-upload-job",
        artifact_uri=f"{BASE_OUTPUT_DIR}/model",
        serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    )
    model_upload_task.after(training_task)

    endpoint_create_task = EndpointCreateOp(
        project=PROJECT_ID,
        display_name=f"{PIPELINE_NAME}-kfp-create-endpoint-job",
    )
    endpoint_create_task.after(model_upload_task)

    model_deploy_op = ModelDeployOp(  # pylint: disable=unused-variable
        model=model_upload_task.outputs["model"],
        endpoint=endpoint_create_task.outputs["endpoint"],
        deployed_model_display_name=MODEL_DISPLAY_NAME,
        dedicated_resources_machine_type=SERVING_MACHINE_TYPE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )
    


Writing ./pipeline_prebuilt_jay_3.py


{
  "studySpec": {
    "metrics": [
      {
        "metricId": "final_val_accuracy",
        "goal": "MAXIMIZE"
      }
    ],
    "parameters": [
      {
        "parameterId": "dropout_rate",
        "doubleValueSpec": {
          "minValue": 0.001,
          "maxValue": 0.4
        },
        "scaleType": "UNIT_LOG_SCALE"
      },
      {
        "parameterId": "l2_regularization_lambda",
        "doubleValueSpec": {
          "minValue": 0.001,
          "maxValue": 0.4
        },
        "scaleType": "UNIT_LOG_SCALE"
      }
    ],
    "measurementSelectionType": "BEST_MEASUREMENT"
  },
  "trialJobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-highmem-16",
          "acceleratorType": "NVIDIA_TESLA_P100",
          "acceleratorCount": 2
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "containerSpec": {
          "imageUri": "gcr.io/qwiklabs-asl-04-76d2f9d7a5ae/trainer_image_ct_phase_contrast_vertex:latest",
          "args": [
            "--train_data_path=gs://capstone-datasets/train_3d.csv",
            "--eval_data_path=gs://capstone-datasets/valid_3d.csv",
            "--output_dir=gs://qwiklabs-asl-04-76d2f9d7a5ae-kfp-artifact-store/models/20230331143721",
            "--hptune"
          ]
        }
      }
    ],
    "baseOutputDirectory": {
      "outputUriPrefix": "gs://qwiklabs-asl-04-76d2f9d7a5ae-kfp-artifact-store/pipeline"
    }
  }
}

In [17]:
### Compile the pipeline

In [170]:
#ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"
ARTIFACT_STORE = f"gs://rutu-bucket"
#PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
PIPELINE_ROOT = f"gs://rutu-bucket"
#DATA_ROOT = f"{ARTIFACT_STORE}/data"
#DATA_ROOT = f"gs://capstone-datasets"
DATA_ROOT = f"gs://rutu-bucket"

#TRAINING_FILE_PATH = f"{DATA_ROOT}/training/dataset.csv"
TRAINING_FILE_PATH = f"{DATA_ROOT}/train_3d.csv"
#VALIDATION_FILE_PATH = f"{DATA_ROOT}/validation/dataset.csv"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/valid_3d.csv"

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BASE_OUTPUT_DIR = f"{ARTIFACT_STORE}/models/{TIMESTAMP}"

#region overide
#REGION='us (multiple regions in United States)'
REGION = "us-central1"


%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}
%env BASE_OUTPUT_DIR={BASE_OUTPUT_DIR}

env: PIPELINE_ROOT=gs://rutu-bucket
env: PROJECT_ID=qwiklabs-asl-04-76d2f9d7a5ae
env: REGION=us-central1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-8:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-asl-04-76d2f9d7a5ae/rutu_image_ct_phase_contrast_kfp_3:latest
env: TRAINING_FILE_PATH=gs://rutu-bucket/train_3d.csv
env: VALIDATION_FILE_PATH=gs://rutu-bucket/valid_3d.csv
env: BASE_OUTPUT_DIR=gs://rutu-bucket/models/20230407135739


In [171]:
#!gsutil ls -R gs://capstone-datasets/valid_3d.csv

Let us make sure that the `ARTIFACT_STORE` has been created, and let us create it if not:

In [172]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://rutu-bucket/


#### Use the CLI compiler to compile the pipeline

In [173]:
PIPELINE_JSON = "rutu_ct_pipeline_new1.json"
PIPELINE_JSON


'rutu_ct_pipeline_new1.json'

In [174]:
#!cat pipeline_prebuilt.py

In [175]:
!dsl-compile-v2 --py pipeline_prebuilt_jay_3.py --output $PIPELINE_JSON

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


**Note:** You can also use the Python SDK to compile the pipeline:

```python
from kfp.v2 import compiler

compiler.Compiler().compile(
    pipeline_func=create_pipeline, 
    package_path=PIPELINE_JSON,
)

```

In [176]:
!head {PIPELINE_JSON}

{
  "pipelineSpec": {
    "components": {
      "comp-custom-training-job": {
        "executorLabel": "exec-custom-training-job",
        "inputDefinitions": {
          "parameters": {
            "base_output_directory": {
              "type": "STRING"
            },


### Deploy the pipeline package

In [177]:
aiplatform.init(project=PROJECT_ID, location=REGION)

pipeline = aiplatform.PipelineJob(
    display_name="rutu_ct_pipeline_3",
    template_path=PIPELINE_JSON,
    enable_caching=True,
)

pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/146339387885/locations/us-central1/pipelineJobs/rutu-ct-pipeline-3-kfp-pipeline-20230407135802
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/146339387885/locations/us-central1/pipelineJobs/rutu-ct-pipeline-3-kfp-pipeline-20230407135802')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/rutu-ct-pipeline-3-kfp-pipeline-20230407135802?project=146339387885
PipelineJob projects/146339387885/locations/us-central1/pipelineJobs/rutu-ct-pipeline-3-kfp-pipeline-20230407135802 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/146339387885/locations/us-central1/pipelineJobs/rutu-ct-pipeline-3-kfp-pipeline-20230407135802 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/146339387885/locations/us-central1/pipelineJobs/rutu-ct-pipeline-3-kfp-pipeline-20230407135802 current state:
PipelineStat

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [model-deploy].; Job (project_id = qwiklabs-asl-04-76d2f9d7a5ae, job_id = 4193276214326067200) is failed due to the above error.; Failed to handle the job: {project_number = 146339387885, job_id = 4193276214326067200}"
